<a href="https://colab.research.google.com/github/mayanku1111/Character-LLM/blob/main/Inference_musk_llm2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title
from huggingface_hub import login
login()

In [ ]:
# @title
!pip install -q gradio
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import random
import json
from datetime import datetime

model_path = "whitepenguin/llama_elon_character"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

elon_profile = {
    "name": "Elon Musk",
    "traits": ["visionary", "ambitious", "technical", "optimistic", "workaholic"],
    "background": "Founder of SpaceX and Tesla, focused on advancing space exploration and sustainable energy",
    "goals": ["Colonize Mars", "Make life multi-planetary", "Advance sustainable technology"],
    "speech_patterns": ["Actually,", "To be frank,", "The future of humanity is...", "It's quite simple:"],
    "knowledge_areas": ["rocket science", "electric vehicles", "solar energy", "artificial intelligence"]
}

def load_chat_history():
    try:
        with open('chat_history.json', 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return []

def save_chat_history(history):
    with open('chat_history.json', 'w') as f:
        json.dump(history, f)

chat_history = load_chat_history()

def generate_response(prompt, max_new_tokens, temperature=0.7, context=""):
    full_prompt = f"[INST] <<SYS>>\nYou are roleplaying as {elon_profile['name']}. Your traits are {', '.join(elon_profile['traits'])}. Your background: {elon_profile['background']}. Your main goals are {', '.join(elon_profile['goals'])}. You have expertise in {', '.join(elon_profile['knowledge_areas'])}. Here's the context of previous conversations:\n\n{context}\n\nNow, respond to the following in character:\n\n{prompt}\n<</SYS>>\n\nProvide a response without including a follow-up question. [/INST]"

    num_prompt_tokens = len(tokenizer(full_prompt)['input_ids'])
    max_length = num_prompt_tokens + max_new_tokens

    gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=2000, temperature=0.7, top_p=0.9, repetition_penalty=1.1,device=0)
    result = gen(full_prompt)
    return result[0]['generated_text'].replace(full_prompt, '')

def generate_follow_up(context):
    follow_up_prompt = f"[INST] <<SYS>>\nYou are roleplaying as {elon_profile['name']}. Based on the following conversation context, generate a relevant follow-up question to continue the discussion about Mars colonization:\n\n{context}\n<</SYS>>\n\nGenerate only the follow-up question, without any additional text. [/INST]"

    gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=2000, temperature=0.7, top_p=0.9, repetition_penalty=1.1,device=0)
    result = gen(follow_up_prompt)
    return result[0]['generated_text'].replace(follow_up_prompt, '').strip()

def apply_character_quirks(response):
    if random.random() < 0.3:
        pattern = random.choice(elon_profile['speech_patterns'])
        response = f"{pattern} {response}"

    if not any(area in response.lower() for area in elon_profile['knowledge_areas']):
        area = random.choice(elon_profile['knowledge_areas'])
        response += f" Of course, this ties into my work with {area}."

    return response

def elon_mars_chat(message, history):
    global chat_history

    recent_context = "\n".join([f"{entry['role']}: {entry['content']}" for entry in chat_history[-5:]])

    response = generate_response(message, max_new_tokens=2000, context=recent_context)
    response = apply_character_quirks(response)

    chat_history.append({"role": "User", "content": message, "timestamp": str(datetime.now())})
    chat_history.append({"role": "Elon Musk", "content": response, "timestamp": str(datetime.now())})

    updated_context = "\n".join([f"{entry['role']}: {entry['content']}" for entry in chat_history[-6:]])
    follow_up = generate_follow_up(updated_context)

    save_chat_history(chat_history)

    formatted_response = f"{response}\n\n {follow_up}?"

    return formatted_response

iface = gr.ChatInterface(
    fn=elon_mars_chat,
    title="Chat with Elon Musk about Mars Colonization",
    description="Engage in a conversation with a simulated Elon Musk about Mars colonization. Each response will include a follow-up question to continue the discussion."
)

iface.launch(debug=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.5 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://281cba269830febc75.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


**Loading previous chats for context**